<a href="https://colab.research.google.com/github/Djaxis/MY-Python-Evolution/blob/main/COLLECT_DATA_STATION_VELO_NANTES_TOUTES_LES_1_MINUTES_Brice_De_Campos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

vérifier que vous avez bien accès à votre compte Google Drive depuis le notebook en exécutant la commande !ls /content/drive/MyDrive pour voir les fichiers présents dans votre Drive.

In [4]:
!ls /content/drive/MyDrive/BDD/

DataNantes2023.csv  UENG_metadata.csv  UENG_metadata.gsheet


In [ ]:
from google.colab import drive
import os.path
import pandas as pd
import csv
import json
import numpy as np
import plotly.express as px
import requests
import seaborn as sns
import time
import urllib.request
import matplotlib.pyplot as plt

# monter votre Google Drive
drive.mount('/content/drive')
# vérifier si le dossier 'BDD' existe dans votre Google Drive
bdd_path = '/content/drive/MyDrive/BDD'
if not os.path.exists(bdd_path):
        os.mkdir(bdd_path)

# vérifier si le fichier 'DataNantes2023.csv' existe dans le dossier 'BDD' de votre Google Drive
csv_path = '/content/drive/MyDrive/BDD/DataNantes2023.csv'
if not os.path.exists(csv_path):
        # si le fichier n'existe pas, créer un nouveau dataframe avec le nom "DataNantes2023"
        DataNantes2023 = pd.DataFrame()
        # enregistrer le dataframe dans le fichier CSV
        DataNantes2023.to_csv(csv_path, index=False)
        print(f'Le fichier {csv_path} a été créé.')
else:
        print(f'Le fichier {csv_path} existe déjà.')

#def collect_data(): 
       # Collecter les données
while True:
                link = 'https://data.nantesmetropole.fr/api/records/1.0/search/?dataset=244400404_stations-velos-libre-service-nantes-metropole-disponibilites&q=&facet=banking&facet=bonus&facet=status&facet=contract_name'
                r = requests.get(link)
                data = json.loads(r.text)

                df_new = pd.json_normalize(data['records']) # pour comprendre et prendre la partie de la data souhaitée. Puis on normalise pour avoir un tableau

                dfviz= df_new[['fields.status','fields.name','fields.address','fields.available_bikes','fields.available_bike_stands','fields.bike_stands','fields.last_update','fields.position']]
                # pour voir les colonnes qui me sont nécessaires et les placer dans un ordre précis

                #renommer les colonnes
                dfrenam = dfviz.rename(columns={'fields.status':'Status',
                                            'fields.name':'Nom Station',
                                            'fields.address':'Adresse De Station',
                                            'fields.available_bikes':'Vélos Disponibles',
                                            'fields.available_bike_stands':'Socles Disponbles',
                                            'fields.bike_stands':'Nombre de Socles',
                                            'fields.last_update':'Dernière Mise à Jour',
                                            'fields.position':'Localisation',
                                            })

                #création de la ligne taux de remplissage
                dfrenam['taux de remplissage'] = dfrenam['Vélos Disponibles']/dfrenam['Nombre de Socles']*100

                # Convertir la colonne en type datetime
                dfrenam['Dernière Mise à Jour'] = pd.to_datetime(dfrenam['Dernière Mise à Jour'], format='%Y-%m-%dT%H:%M:%S+00:00')

                # Séparer la colonne en deux colonnes distinctes date et heure
                dfrenam['date'] = dfrenam['Dernière Mise à Jour'].dt.date
                dfrenam['heure'] = dfrenam['Dernière Mise à Jour'].dt.time

                # Arrondir la colonne 'taux de remplissage' à deux décimales
                dfrenam['taux de remplissage'] = dfrenam['taux de remplissage'].round(2)
          
                # Ajouter les nouvelles données au dataframe existant
                #dfactu = pd.concat([DataNantes2023, dfrenam])

                # Sauvegarder le dataframe dans le fichier CSV
                dfrenam.to_csv('/content/drive/MyDrive/BDD/DataNantes2023.csv', index=False, mode='a')
                
                #Vérification de la présence du fichier.
                !ls /content/drive/MyDrive/BDD/

                # vérification visuelle de la ré-écriture dans le fichier.
                lire = pd.read_csv(r"/content/drive/MyDrive/BDD/DataNantes2023.csv", header=None,sep = ",")
                lire = pd.DataFrame(lire)#convertir le sheet en data
                
                print("NOMBRE DE LIGNES PRESENTES:-", len(lire))
                print(lire)

                # Attendre 1 minutes avant de collecter de nouvelles données
                time.sleep(60)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Le fichier /content/drive/MyDrive/BDD/DataNantes2023.csv existe déjà.
DataNantes2023.csv  UENG_metadata.csv  UENG_metadata.gsheet
NOMBRE DE LIGNES PRESENTES:- 363
         0                      1   \
0    Status            Nom Station   
1      OPEN             084-RÉGION   
2      OPEN           104-ANTILLES   
3      OPEN              090-PROCÉ   
4      OPEN   017-SAINTE ÉLISABETH   
..      ...                    ...   
358    OPEN        115-TRENTEMOULT   
359    OPEN     002-HÔTEL DE VILLE   
360    OPEN             018-VIARME   
361    OPEN  044-PALAIS DE JUSTICE   
362    OPEN            050-CHÂTEAU   

                                                    2                  3   \
0                                   Adresse De Station  Vélos Disponibles   
1              Rue de la Loire - place Gaston Defferre                  8   
2                   